In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

In [2]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
#总计20640个样本，每个样本8个属性表示，以及房价作为target，所有属性值均为number
#目标变量：平均房屋价值
#输入变量（特征）：平均收入、住房平均年龄、平均房间、平均卧室、人口、平均占用、纬度和经度

X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target.reshape(-1, 1), random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)

In [3]:
input_shape = X_train.shape[1:]

In [4]:
#创建一个函数重新创建已配置的损失函数，阈值默认值为1.0
def create_huber(threshold=1.0):
    def huber_fn(y_true, y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error) / 2
        linear_loss  = threshold * tf.abs(error) - threshold**2 / 2
        return tf.where(is_small_error, squared_loss, linear_loss)
    return huber_fn

### 基于模型内部的损失和指标

模型内部的指标如权重或隐藏层的激活对于正则化或见识模型的某些内部方面可能很有用

要基于模型内部自定义损失，根据所需模型的任何部分进行计算，然后将结果传递给add_loss()方法

#### 自定义重建损失的自定义模型

重建损失：辅助输出有关的损失，他是重建与输入之间的均方差

#### tf.keras.metrics.Mean(name="mean", dtype=None):计算给定值的（加权）平均值。

name:（可选）度量标准实例的字符串名称。
dtype:（可选）度量标准结果的数据类型。

如果 sample_weight 为 None ，则权重默认为1。使用 sample_weight 为0掩盖值。

#### add_loss（）：将损失添加到模型的损失列表中

### add_metric()：将指标添加到模型的指标列表中

In [7]:
class ReconstructingRegressor(keras.Model):
    """
    构建具有5个密集隐藏层和1个密集输出层的DNN；
    通过将重建损失添加到主要损失中，
    模型可通过隐藏层保留尽可能多的信息，实际应用时有时会提高泛化性
    """
    def __init__(self, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.hidden = [keras.layers.Dense(30, activation="selu",
                          kernel_initializer="lecun_normal")
                       for _ in range(5)]
        self.out = keras.layers.Dense(output_dim)
        #重建损失指标
        self.reconstruction_mean = keras.metrics.Mean(name="reconstruction_error")

    def build(self, batch_input_shape):
        """
        创建一个额外的密集层，用于重建模型的输入（定义模型内部的损失）
        其单元数（神经元个数）必须=数据集输入的label种类数
        """
        n_inputs = batch_input_shape[-1]
        #print("batch_input_shape[-1]",batch_input_shape[-1])#8
        self.reconstruct = keras.layers.Dense(n_inputs)#创建一个密集层

    
    def call(self, inputs, training=None):
        """处理5个隐藏层中每个层的输入，将输出结果传递到重建层产生重构"""
        #构造使用模型
        Z = inputs
        for layer in self.hidden:
            Z = layer(Z)  
        reconstruction = self.reconstruct(Z)#重构层的输出
        
        #重建损失 = 重构与输入之间的均方差
        recon_loss = tf.reduce_mean(tf.square(reconstruction - inputs))

        #保存结果（按比例缩减使得重建损失不在主要损失中占大部分，0.05是可调整的）
        self.add_loss(0.05 * recon_loss)
        
        if training:#训练结束后重建损失不变
            #计算recon_loss的加权平均值，作为一个自定义指标
            result = self.reconstruction_mean(recon_loss)
            self.add_metric(result)#保存结果，将自定义指标添加到模型中
            
        return self.out(Z)#返回输出层的输出

In [8]:
model = ReconstructingRegressor(1)
#输出维度为1
model.compile(loss="mse", optimizer="nadam")
history = model.fit(X_train_scaled, y_train, epochs=2)
#这里的loss是每个轮次的平均损失（主要损失+0.05倍的重建损失）
#reconstruction_error是自定义的指标：每个轮次的重建误差

Epoch 1/2
363/363 [==============================] - 2s 2ms/step - loss: 0.6879 - reconstruction_error: 0.8997
Epoch 2/2
363/363 [==============================] - 1s 2ms/step - loss: 0.4061 - reconstruction_error: 0.3864


In [9]:
y_pred = model.predict(X_test_scaled)
y_pred

162/162 [==============================] - 0s 1ms/step


array([[0.19275814],
       [1.3830202 ],
       [4.2713537 ],
       ...,
       [1.4996471 ],
       [2.6838899 ],
       [4.256939  ]], dtype=float32)